In [1]:
#pip install --upgrade pip
#pip install beautifulsoup4

# Сбор итемов

In [3]:
import requests
from bs4 import BeautifulSoup
import threading
from src.const_parameters import *
from database_torrentitem import *

def get_identifiers_from_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        div_values = []
        for div in soup.find_all('h3', {'class':'h2 m0'}):
            style = div.attrs['style']
            if(style.find("view-transition-name: title-") > -1):
                data = TorrentItem()
                data.guid = style.removeprefix("view-transition-name: title-")
                data.name = div.getText()
                div_values.append(data)
        return div_values
    else:
        print(f'[{response.status_code}] {response.text}')
        return None
    
def collectBaseItemsFromAddress(SITE_ADDRESS, CATEGORY):
    result = {}
    pages = {}
    def get_identifiers_from_page_async(iPage):
        page = get_identifiers_from_page(f"{SITE_ADDRESS}/torrents/{CATEGORY}/?page={iPage}")
        result[iPage] = not (page == None or len(page) < 1)
        if(result[iPage] == True):
            pages[iPage] = page

    iPage = 1
    while True:
        threads = []
        for iThread in range(THREAD_MAX):
                t = threading.Thread(target=get_identifiers_from_page_async, args=([iPage + iThread]))
                t.start()
                threads.append(t)

        for t in threads:
            t.join()

        if(all(result.values()) == False):
            break

        iPage = iPage + THREAD_MAX
    return pages

# Размещение итемов в папках

In [5]:
import os

In [6]:
def createFolderIfNotExist(s):
    if not os.path.exists(s):
        os.makedirs(s)

In [7]:
def createFilesDataBase(category, pages):
    createFolderIfNotExist(FOLDER_DATABASE)
    categoryFolder = os.path.join(FOLDER_DATABASE, category)

    createFolderIfNotExist(categoryFolder)

    for key in pages:
        for item in pages[key]:
            itemBasePath = os.path.join(categoryFolder, item.guid)
            if os.path.exists(itemBasePath):
                itemInDB = TorrentItem.from_json(itemBasePath)
                itemInDB.compareWith(item)
            else:
                item.to_json(itemBasePath + ".json")

# Выполнение

In [8]:
def updateOrCreateDataBaseFiles(address, category:str):
    pages = collectBaseItemsFromAddress(address, category)
    category = category.replace('/', '_')
    createFilesDataBase(category, pages)

In [9]:
if(__name__ == "__main__"):
    updateOrCreateDataBaseFiles(BASE_ADDRESS_URI, "anime")
